In [ ]:
import boto3

import rasterio
import rasterio.transform
from rasterio.session import AWSSession
from rasterio.windows import Window, from_bounds

from dataplug import CloudObject
from dataplug.formats.geospatial.cog import CloudOptimizedGeoTiff, grid_partition_strategy

session = boto3.Session()
creds = session.get_credentials().get_frozen_credentials()

s3_config = {
        "credentials": {
                "AccessKeyId": creds.access_key,
                "SecretAccessKey": creds.secret_key,
                "SessionToken": creds.token,
        },
        "region_name": "us-east-1"
}

In [ ]:
# Replace by your ID
my_id = input()
print("Your ID is", my_id)

In [ ]:
cog_url = "s3://sentinel-cogs/sentinel-s2-l2a-cogs/1/C/CV/2023/1/S2B_1CCV_20230111_0_L2A/B02.tif"
co = CloudObject.from_s3(CloudOptimizedGeoTiff, 
                         cog_url, 
                         s3_config=s3_config,
                         metadata_bucket=f"scipy-tutorial-meta-{str(my_id)}")


In [ ]:
# Data must be pre-processed first ==> This only needs to be done once per dataset
# Preprocessing will create reusable indexes to repartition
# the data many times in many chunk sizes
co.preprocess()

# Partition the object
# This does not move data around, it only creates data slices from the indexes
data_slices = co.partition(grid_partition_strategy, n_splits=2)

for idx, data_slice in enumerate(data_slices):
        # Evaluate the data_slice, which will perform the
        # actual HTTP GET requests to get the partition data
        chunk_data = data_slice.get()
        print(chunk_data)
        print("-------------")